###From drive to youtube video upload

####Required packages

In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

     |████████████████████████████████| 7.2 MB 15.8 MB/s 
  Attempting uninstall: google-auth-httplib2
    Found existing installation: google-auth-httplib2 0.0.4
    Uninstalling google-auth-httplib2-0.0.4:
      Successfully uninstalled google-auth-httplib2-0.0.4
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.4
    Uninstalling google-auth-oauthlib-0.4.4:
      Successfully uninstalled google-auth-oauthlib-0.4.4
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.272 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 2.15.0 which is 

#### Copying file from drive to colab

In [2]:
!cp '/content/drive/MyDrive/Comp final BIGGER.mp4' '/content/'

####Now uploading file to youtube

 *'#' in following code notifies important changes*

In [10]:
import datetime
from googleapiclient.http import MediaFileUpload
import pickle
import os
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.auth.transport.requests import Request
import datetime

CLIENT_SECRET_FILE = '/content/client_secret.json'  #
API_NAME = 'youtube'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/youtube.upload']



def Create_Service(client_secret_file, api_name, api_version, *scopes):
    print(client_secret_file, api_name, api_version, scopes, sep='-')
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes[0]]
    print(SCOPES)

    cred = None

    pickle_file = f'token_{API_SERVICE_NAME}_{API_VERSION}.pickle'
    # print(pickle_file)

    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as token:
            cred = pickle.load(token)

    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            cred = flow.run_local_server()

        with open(pickle_file, 'wb') as token:
            pickle.dump(cred, token)

    try:
        service = build(API_SERVICE_NAME, API_VERSION, credentials=cred)
        print(API_SERVICE_NAME, 'service created successfully')
        return service
    except Exception as e:
        print('Unable to connect.')
        print(e)
        return None

def convert_to_RFC_datetime(year=1900, month=1, day=1, hour=0, minute=0):
    dt = datetime.datetime(year, month, day, hour, minute, 0).isoformat() + 'Z'
    return dt


service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)

upload_date_time = datetime.datetime(2021, 8, 7, 2, 3, 0).isoformat() + '.000Z' #

request_body = {
    'snippet': {
        'categoryI': 19,
        'title': "Brother's wedding invitation",
        'description': 'sdasdHello World Description',
        'tags': ['Travel', 'video test', 'Travel Tips','wedding invitation']
    },
    'status': {
        'privacyStatus': 'public', #
        # 'publishAt': upload_date_time,  #this only works for private videos
        'selfDeclaredMadeForKids': False, 
    },
    'notifySubscribers': False #
}

mediaFile = MediaFileUpload('/content/comp_final_bigger.mp4')  ##

response_upload = service.videos().insert(
    part='snippet,status',
    body=request_body,
    media_body=mediaFile
).execute()


service.thumbnails().set(
    videoId=response_upload.get('id'),
    media_body=MediaFileUpload('thumbnail.png')   ##
).execute()

/content/client_secret.json-youtube-v3-(['https://www.googleapis.com/auth/youtube.upload'],)
['https://www.googleapis.com/auth/youtube.upload']
youtube service created successfully


{'etag': 'MA12JHmh6aG4-41GvbpMJLIXlNA',
 'items': [{'default': {'height': 90,
    'url': 'https://i.ytimg.com/vi/PP3fKIaAdnM/default.jpg',
    'width': 120},
   'high': {'height': 360,
    'url': 'https://i.ytimg.com/vi/PP3fKIaAdnM/hqdefault.jpg',
    'width': 480},
   'maxres': {'height': 720,
    'url': 'https://i.ytimg.com/vi/PP3fKIaAdnM/maxresdefault.jpg',
    'width': 1280},
   'medium': {'height': 180,
    'url': 'https://i.ytimg.com/vi/PP3fKIaAdnM/mqdefault.jpg',
    'width': 320},
   'standard': {'height': 480,
    'url': 'https://i.ytimg.com/vi/PP3fKIaAdnM/sddefault.jpg',
    'width': 640}}],
 'kind': 'youtube#thumbnailSetResponse'}